# Global data on the sphere

<div class="alert alert-block alert-info"> ETOPO1 is a 1 arc-minute global relief model of Earth's surface that integrates land topography and ocean bathymetry. Built from global and regional data sets, it is available in "Ice Surface" (top of Antarctic and Greenland ice sheets) and "Bedrock" (base of the ice sheets). 

In this example, we read the global topography database ETOPO1 that can be downloaded from NOAA website: 
**https://www.ngdc.noaa.gov/mgg/global/global.html**</div>

***

<img src="images/sphere.png" width="50%">

***

## Notebook contents

   - [ETOPO1 GeoTIFF](#ETOPO1-GeoTIFF)
   - [Triangulation on the sphere](#Triangulation-on-the-sphere)
   - [Mapping dataset on the sphere](#Mapping-dataset-on-the-sphere)
   - [Visualisation](#Visualisation)

In [ ]:
import gdal
import meshio
import numpy as np
import os.path as path
import stripy as stripy
from scipy import ndimage

# ETOPO1 GeoTIFF


<div class="alert alert-block alert-danger"> To run this Notebook you will first need to download the ETOPO1 dataset following the link provided above...</div>

GeoTIFF is the most versatile raster format for geo-referenced imagery and is the default for all `gdal` utilities.

We load the *downloaded GeoTiff*...

In [ ]:
gtiff = gdal.Open("data/ETOPO1.tif")

width = gtiff.RasterXSize
height = gtiff.RasterYSize
gt = gtiff.GetGeoTransform()
img = gtiff.GetRasterBand(1).ReadAsArray().T
img = np.fliplr(img)

# Triangulation on the sphere

We use [**`stripy`**](https://github.com/University-of-Melbourne-Geodynamics/stripy), that provides a python interfact to **STRIPACK** and **SSRFPACK** (Renka 1997a,b) as a triangulation/spherical class.

Here we use a specific class from the library: `spherical_meshes.icosahedral_mesh` to build our mesh based on the recursive refinement of an initial icosahedron. The higher the refinement the higher the resolution will be...

#### References
 
   1. Renka, R. J. (1997), Algorithm 772: STRIPACK: Delaunay triangulation and Voronoi diagram on the surface of a sphere, ACM Transactions on Mathematical Software (TOMS).
   
   2. Renka, R. J. (1997), Algorithm 773: SSRFPACK: interpolation of scattered data on the surface of a sphere with a surface under tension, ACM Transactions on Mathematical Software (TOMS), 23(3), 435–442, doi:10.1145/275323.275330.

In [ ]:
grid = stripy.spherical_meshes.icosahedral_mesh(include_face_points=False, refinement_levels=8)

str_fmt = "{:25} {:9}"
print(str_fmt.format('Number of points', grid.npoints))
print(str_fmt.format('Number of cells', grid.simplices.shape[0]))

# Mapping dataset on the sphere

Now we map the ETOPO1 elevation to the spherical mesh by interpolation using **scipy** [**`map_coordinates`**](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.ndimage.interpolation.map_coordinates.html.)

Our spherical mesh is defined on the unit sphere and the longitudes and latitudes range from [$-\pi,\pi$] and [-$\pi/2,\pi/2$] respectively. 

We first convert the points in degrees and map their position with respect to ETOPO1 dataset shape.

In [ ]:
# Convert spherical mesh longitudes and latitudes to degrees
dlons = np.mod(np.degrees(grid.lons)+180.0, 360.0)
dlats = np.mod(np.degrees(grid.lats)+90, 180.0)

# Map mesh coordinates on ETOPO1 dataset
ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0

icoords = np.stack((ilons, ilats))

We then performe the interpolation...

In [ ]:
elevations = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)

Finally we set the spherical mesh coordinates in meters assuming a _spherical Earth_ with a radius of $\sim$6378 km and adding the previously interpolated ETOPO1 elevation along the Z axis...

In [ ]:
x = grid.points[:,0]*6378137.
y = grid.points[:,1]*6378137. 
h = grid.points[:,2]*6378137.+ elevations 

coords = np.vstack((x,y))
coords = np.vstack((coords,h)).T

# Visualisation

## Paraview

With [`meshio`](https://github.com/nschloe/meshio), we can write the dataset and visualise it in [**Paraview**](https://www.paraview.org).

You can then use a `Calculator` to increase the vertical exaggeration as shown in the figure below:

<img src="images/calculator.png" width="50%">

$$(iHat * coordsX + jHat * coordsY + kHat * coordsZ) * ((1 + (Z/EarthRadius) * Zex))$$

***

Here we chose to export the mesh as a `VTK` file called _globe.vtk_ and this is done like this: 

In [ ]:
mesh = meshio.Mesh(coords, {'triangle': grid.simplices}, {'Z':elevations})
meshio.write("data/globe.vtk", mesh)

## LavaVu

Another option is to use [**LavaVu**](https://github.com/OKaluza/LavaVu) a scientific visualisation tool with a python interface for fast and flexible visual analysis.

***

<div class="alert alert-block alert-danger"> Be aware that depending of the size of your dataset, this might take a while to render...</div>

In [ ]:
import lavavu

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[500,500], near=-10.0)

# Core 
tris2 = lv.triangles("triangles",  wireframe=False, colour="#77ff88", opacity=1.0)
tris2.vertices(grid.points * (1.0+0.000003*elevations.reshape(-1,1)))
tris2.indices(grid.simplices)
tris2.values(elevations*0.001)
tris2.colourmap("(-5.0)#3333FF:0.85 (-0.001)#004477 (0.0)#448822 (0.1)#BB9911 (1.0)#B05030 (5.0)White" , logscale=False, range=[-7.0,5.0])   # Apply a built in colourmap

lv.translation(0.0, 0.0, -3.329)
lv.rotation(122.06, 41.123, -157.397)

# Let this be an interactive window in the notebook
lv.window()
tris2.control.Checkbox(property='wireframe', label="Surface - wireframe")

lv.control.Range('specular', range=(0,1), step=0.1, value=0)
lv.control.Checkbox(property='axis')
lv.control.ObjectList()
lv.control.show()